In [1]:
!git clone https://github.com/usman9-ai/Virtual-Try-On.git
%cd CatVTON

Cloning into 'Virtual-Try-On'...
remote: Enumerating objects: 785, done.
remote: Counting objects: 100% (785/785), done.
remote: Compressing objects: 100% (661/661), done.
remote: Total 785 (delta 110), reused 780 (delta 105), pack-reused 0 (from 0)
Receiving objects: 100% (785/785), 16.06 MiB | 7.43 MiB/s, done.
Resolving deltas: 100% (110/110), done.
[Errno 2] No such file or directory: 'CatVTON'
/kaggle/working


In [2]:
!pip install -r "/kaggle/working/Virtual-Try-On/requirements.txt"
!pip install pyngrok
# !pip install -q diffusers transformers accelerate huggingface-hub gradio tqdm
!pip install -q torch torchvision --extra-index-url https://download.pytorch.org/whl/cu113
!pip install -q diffusers transformers accelerate huggingface-hub gradio tqdm flask pyngrok requests

  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-t48as6p4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-t48as6p4
  Resolved https://github.com/huggingface/diffusers.git to commit 6508da6f06a0da1054ae6a808d0025c04b70f0e8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of diffusers to dete

In [3]:
!pip install fvcore av 

  Using cached fvcore-0.1.5.post20221221.tar.gz (50 kB)
  Preparing metadata (setup.py) ... done
  Using cached yacs-0.1.8-py3-none-any.whl.metadata (639 bytes)
  Using cached iopath-0.1.10.tar.gz (42 kB)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 54.7 MB/s eta 0:00:00:00:0100:01
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=db59fc0921f824a49a02738dfaeea3037a93b3d953769b9b78e0ea61ef61df11
  Stored in directory: /root/.cache/pip/wheels/65/71/95/3b8fde5c65c6e4a806e0867c1651dcc71a1cb2f3430e8f355f
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=678853edcc9a1ea7ea90e345daf5b75d95f4c00c397c4c940e91e8052889c053
  Stored in directory: /root/.cache/pip/wheels/ba/5e/16/6117f8fe7e9c0c161a795e10d94645ebcf301ccbd01f66d8ec
Successfully built fvcore iopath


In [4]:
%cd "/kaggle/working/Virtual-Try-On"

/kaggle/working/Virtual-Try-On


In [ ]:
# CatVTON Inference API with URL Support and Ngrok Endpoint

# 1. Install necessary packages
# !pip install -q diffusers transformers accelerate huggingface-hub gradio tqdm flask requests

# 2. Import required libraries
!pip install cloudinary
import os
import io
import torch
import requests
from PIL import Image
from flask import Flask, request, jsonify
from diffusers.image_processor import VaeImageProcessor
from huggingface_hub import snapshot_download
import numpy as np
from pyngrok import ngrok


# 3. Import CatVTON-specific modules
from model.pipeline import CatVTONPipeline
from model.cloth_masker import AutoMasker
from utils import resize_and_crop, resize_and_padding, init_weight_dtype


os.environ['CLOUDINARY_CLOUD_NAME'] = "df3aqiyu5"
os.environ['CLOUDINARY_API_KEY'] = "195584773642988"
os.environ['CLOUDINARY_API_SECRET'] = "LAQuEUTe1Cg6-O80E-BOuPbogjk"

# 4. Clear CUDA cache
torch.cuda.empty_cache()

# 5. Download and initialize model
repo_ckpt = "zhengchong/CatVTON"
attn_folder = snapshot_download(repo_id=repo_ckpt)

pipeline = CatVTONPipeline(
    base_ckpt="runwayml/stable-diffusion-inpainting",
    attn_ckpt=attn_folder,
    attn_ckpt_version="mix",
    weight_dtype=init_weight_dtype("fp16"),
    use_tf32=True,
    device='cuda',
    skip_safety_check=True
)

# 6. Setup AutoMasker and VAE mask processor
mask_processor = VaeImageProcessor(
    vae_scale_factor=8,
    do_normalize=False,
    do_binarize=True,
    do_convert_grayscale=True
)
automasker = AutoMasker(
    densepose_ckpt=os.path.join(attn_folder, "DensePose"),
    schp_ckpt=os.path.join(attn_folder, "SCHP"),
    device='cuda'
)

# 7. Function to download image from URL
def download_image_from_url(url):
    try:
        response = requests.get(url, stream=True, timeout=10)
        response.raise_for_status()
        img = Image.open(io.BytesIO(response.content)).convert('RGB')
        return img
    except Exception as e:
        print(f"Error downloading image from {url}: {e}")
        return None

# 8. Function to upload image to Cloudinary
def upload_to_cloudinary(image):
    import cloudinary
    import cloudinary.uploader
    
    # Configure Cloudinary - replace with your credentials
    cloudinary.config(
        cloud_name=os.environ.get('CLOUDINARY_CLOUD_NAME'),
        api_key=os.environ.get('CLOUDINARY_API_KEY'),
        api_secret=os.environ.get('CLOUDINARY_API_SECRET')
    )
    
    # Save image to a temporary buffer
    buffer = io.BytesIO()
    image.save(buffer, format='PNG')
    buffer.seek(0)
    
    # Upload to Cloudinary
    try:
        response = cloudinary.uploader.upload(
            buffer,
            folder="virtual-try-on/results",
            resource_type="image"
        )
        return response['secure_url']
    except Exception as e:
        print(f"Error uploading to Cloudinary: {e}")
        return None

# 9. Main try-on function that accepts URLs
def run_tryon_from_urls(person_url, cloth_url, cloth_type='upper', steps=25, scale=0.5, seed=42):
    try:
        # Download images from URLs
        print(cloth_type)
        person = download_image_from_url(person_url)
        cloth = download_image_from_url(cloth_url)
        
        if person is None or cloth is None:
            return {"error": "Failed to download images from provided URLs"}, None
        
        # Set resolution
        W, H = pipeline.unet.config.sample_size*8, pipeline.unet.config.sample_size*8
        # W, H = 128,200
        # Preprocess images
        person = resize_and_padding(person, (W, H))
        cloth = resize_and_padding(cloth, (W, H))
        
        # Auto-generate mask
        raw_mask = automasker(person, cloth_type)['mask']
        raw_mask = resize_and_padding(raw_mask, (W, H))
        mask = mask_processor.blur(raw_mask, blur_factor=9)
        mask = mask.convert('L')
        
        # Seed control
        generator = torch.Generator(device='cuda').manual_seed(seed)
        
        # Inference
        images = pipeline(
            image=person,
            condition_image=cloth,
            mask=mask,
            num_inference_steps=steps,
            guidance_scale=scale,
            generator=generator
        )
        
        result_image = images[0]
        
        # Return success response with the result image
        return {"status": "success"}, result_image
    
    except Exception as e:
        print(f"Error in try-on process: {str(e)}")
        return {"error": str(e)}, None

# 10. Create Flask application
app = Flask(__name__)

@app.route('/process-images', methods=['POST'])
def process_images():
    try:
        # Get JSON data from request
        data = request.json
        
        if not data or 'user_image_url' not in data or 'cloth_image_url' not in data:
            return jsonify({"error": "Missing required parameters: user_image_url, cloth_image_url"}), 400
        
        user_image_url = data['user_image_url']
        cloth_image_url = data['cloth_image_url']
        
        # Optional parameters with defaults
        cloth_type = data.get('cloth_type', 'upper')
        print(f"Cloth Type:, {cloth_type}")
        steps = int(data.get('steps', 25))
        scale = float(data.get('scale', 2.5))
        seed = int(data.get('seed', 42))
        
        print(f"Processing try-on for:\nUser image: {user_image_url}\nCloth image: {cloth_image_url}")
        
        # Process the try-on
        result, result_image = run_tryon_from_urls(
            user_image_url, 
            cloth_image_url,
            cloth_type=cloth_type,
            steps=50,
            scale=3.0,
            seed=seed
        )
        
        if result_image:
            # Upload result to Cloudinary
            result_url = upload_to_cloudinary(result_image)
            
            if result_url:
                return jsonify({
                    "status": "success",
                    "result_image_url": result_url
                })
            else:
                return jsonify({"error": "Failed to upload result image"}), 500
        else:
            return jsonify(result), 400
            
    except Exception as e:
        print(f"Error processing request: {str(e)}")
        return jsonify({"error": str(e)}), 500


# 11. Set up Ngrok and start the server
import threading
import time

def run_flask_app():
    app.run(host='0.0.0.0', port=5000)

# Start Flask in a background thread
flask_thread = threading.Thread(target=run_flask_app)
flask_thread.daemon = True
flask_thread.start()

# Give the server a moment to start
time.sleep(5)

# Now set up Ngrok tunnel
ngrok_token = r"2wxO4zm2p15Q7QmipxuabtzhrlD_5rwi3U4fuLCHAj1t8oL6W"
if ngrok_token:
    ngrok.set_auth_token(ngrok_token)

ngrok_tunnel = ngrok.connect(5000)
print(f"Ngrok endpoint URL: {ngrok_tunnel.public_url}")
print("Use this URL as your GPU_ENDPOINT_URL in the FastAPI backend")

# Keep the main thread alive to prevent termination
try:
    while True:
        time.sleep(60)
except KeyboardInterrupt:
    print("Shutting down...")
    ngrok.kill()

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

An error occurred while trying to fetch runwayml/stable-diffusion-inpainting: runwayml/stable-diffusion-inpainting does not appear to have a file named diffusion_pytorch_model.safetensors.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


Ngrok endpoint URL: https://3973-104-196-114-246.ngrok-free.app
Use this URL as your GPU_ENDPOINT_URL in the FastAPI backend
Cloth Type:, upper
Processing try-on for:
User image: https://res.cloudinary.com/df3aqiyu5/image/upload/v1748611966/virtual-try-on/user_5fff08a8-d6bc-403c-a882-69cd4f71962c.png
Cloth image: https://res.cloudinary.com/df3aqiyu5/image/upload/v1748611966/virtual-try-on/cloth_2a664abc-d75e-4347-8bc9-dee4120d8b3f.jpg
upper


100%|██████████| 30/30 [01:13<00:00,  2.46s/it]


Cloth Type:, upper
Processing try-on for:
User image: https://res.cloudinary.com/df3aqiyu5/image/upload/v1748612084/virtual-try-on/user_7f694afe-0932-4663-b876-465e30413506.png
Cloth image: https://res.cloudinary.com/df3aqiyu5/image/upload/v1748612085/virtual-try-on/cloth_3e6ed0cf-2d87-4352-85f4-b64aebf48e0b.jpg
upper


100%|██████████| 30/30 [01:13<00:00,  2.46s/it]
